# Route Identification Over Time, Approach 1

Recent observations shows small chages in routes over time. Specifically in the following fields:
* route ID
* route short name
* route long name
* route desc

Need to observe these route changes in order to account for these changes in future analyses.

## Objective
1. Query data from `fct_monthly_routes` to help identify variences in Routes. Query for 2023, a couple of months. 
2. Save data to GCS `gtfs_schedule` bucket
3. Filter down data to `Sacramento Regional Transit`, identify and observe routes for any variences

* https://github.com/cal-itp/data-analyses/issues/924

In [ ]:
import geopandas as gpd
import pandas as pd
from shared_utils import rt_dates
from calitp_data_analysis import geography_utils

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
gdf = gpd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/gtfs_schedule/route_identification_2023_m04_m09.parquet"
)

In [ ]:
gdf.shape

In [ ]:
gdf = gdf.drop(columns=["year", "key"])

In [ ]:
gdf.sample().drop(columns=["geometry"])

### LA Metro

In [ ]:
# gdf.name.unique()

In [ ]:
la_metro = gdf.loc[gdf.name == "LA Metro Bus Schedule"].reset_index(drop=True)

In [ ]:
la_metro.shape

In [ ]:
la_metro.month.unique()

In [ ]:
# Compare each month
# https://stackoverflow.com/questions/47769453/pandas-split-dataframe-to-multiple-by-unique-values-rows
la_dfs = dict(tuple(la_metro.groupby("month")))

In [ ]:
la_metro_df = la_metro.drop(columns = ['geometry'])

In [ ]:
la_metro_df.groupby(['route_id']).agg({'shape_id':'nunique'}).sort_values(['shape_id'], ascending = False).head()

In [ ]:
april = la_dfs[4]

In [ ]:
april.columns

In [ ]:
may = la_dfs[5]

In [ ]:
june = la_dfs[6]

In [ ]:
pd.merge(
    april,
    may,
    on=["name", "source_record_id", "route_id", "shape_id"],
    how="outer",
    indicator=True,
)[["_merge"]].value_counts()

In [ ]:
m1 = (
    pd.merge(
        april,
        may,
        on=["name", "source_record_id", "route_id", "shape_id"],
        how="outer",
        indicator=True,
    )
    .rename(columns={"_merge": "april_v_may"})
    .drop(columns=["month_x", "month_y"])
)

In [ ]:
m1.columns

In [ ]:
preview = ["source_record_id", "name", "route_id", "shape_id", "april_v_may"]

In [ ]:
m1.loc[m1.april_v_may == "left_only"].set_geometry("geometry_x").explore(
    "route_id", style_kwds={"weight": 10}
)

In [ ]:
m1.loc[m1.april_v_may == "right_only"].set_geometry("geometry_y").explore(
    "route_id", style_kwds={"weight": 10}
)

In [ ]:
m1 = m1.set_geometry('geometry_x').to_crs(geography_utils.CA_StatePlane)

In [ ]:
m1['april_geo_len'] = m1.geometry_x.length

In [ ]:
m1 = m1.set_geometry('geometry_y').to_crs(geography_utils.CA_StatePlane)

In [ ]:
m1['may_len'] = m1.geometry_y.length

In [ ]:
m1.loc[m1.april_v_may != "both"][preview]

In [ ]:
may.shape, april.shape

In [ ]:
april.geometry.geom_type.unique()

In [ ]:
may.geometry.geom_type.unique()

In [ ]:
group1 = m1.loc[m1.april_v_may != "both"].groupby(['route_id',]).agg({'shape_id':'nunique', 'april_geo_len':'max', 'may_len':'max'}).reset_index()

In [ ]:
group1.april_geo_len/group1.may_len * 100